In [ ]:
from google.colab import drive
import os
drive.mount('/content/drive')
os.chdir(r"/content/drive/MyDrive/ImageClef-2019-VQA-Med-Full")
os.getcwd()

Mounted at /content/drive


'/content/drive/MyDrive/ImageClef-2019-VQA-Med-Full'

In [ ]:
import glob
import random
import numpy as np
import pandas as pd
import time
import datetime as dt
import matplotlib.pyplot as plt
from mpl_toolkits.axes_grid1 import ImageGrid
from sklearn.preprocessing import LabelEncoder
from os import listdir, makedirs, getcwd, remove
from os.path import isfile, join, abspath, exists, isdir, expanduser
from PIL import Image
import torch
from torch.optim import lr_scheduler
from torch.autograd import Variable
from torch.utils.data import Dataset, DataLoader
import torchvision
from torchvision import transforms, datasets, models
from torch import optim, cuda
from timeit import default_timer as timer

In [ ]:
label_dict = {'C4_Abnormality':3,
             'C3_Organ':2,
             'C2_Plane':1,
             'C1_Modality':0}

TRAIN_FOLDER = '/content/drive/MyDrive/ImageClef-2019-VQA-Med-Full/ImageClef-2019-VQA-Med-Training/ImageClef-2019-VQA-Med-Training/Train_images'
VAL_FOLDER = '/content/drive/MyDrive/ImageClef-2019-VQA-Med-Full/ImageClef-2019-VQA-Med-Validation/ImageClef-2019-VQA-Med-Validation/Val_images'

In [ ]:
path = '/content/drive/MyDrive/ImageClef-2019-VQA-Med-Full/ImageClef-2019-VQA-Med-Training/ImageClef-2019-VQA-Med-Training/QAPairsByCategory'

filenames = glob.glob(path + "/*.txt")

dfs = []

for filename in filenames:
    category = filename.split('/')[-1].replace('_train.txt','')
    df_temp = pd.read_csv(filename, delimiter='|', names=['id','question','answer'])
    df_temp['label'] = label_dict.get(category)
    dfs.append(df_temp)

df_train = pd.concat(dfs, ignore_index=True)

In [ ]:
df_train.head()

,id,question,answer,label
0,synpic41148,what kind of image is this?,cta - ct angiography,0
1,synpic43984,is this a t1 weighted image?,no,0
2,synpic38930,what type of imaging modality is used to acqui...,us - ultrasound,0
3,synpic52143,is this a noncontrast mri?,no,0
4,synpic20934,what type of image modality is this?,xr - plain film,0


In [ ]:
# Filter df_train to include only 'C3_Organ' and 'C2_Plane' labels
df_train = df_train[(df_train['label'] == 1)]

path = r'ImageClef-2019-VQA-Med-Validation/ImageClef-2019-VQA-Med-Validation/QAPairsByCategory'

# Get a list of all .txt files in the specified path
filenames = glob.glob(path + "/*.txt")

dfs = []

# Iterate through each file
for filename in filenames:
    # Extract the label from the filename
    label = filename.split('/')[-1].replace('_val.txt', '')
    # Ensure label is in label_dict
    if label in label_dict:
        # Read the text file into a DataFrame with specified delimiter and column names
        df_temp = pd.read_csv(filename, delimiter='|', names=['id', 'question', 'answer'])
        # Add the label to the DataFrame
        df_temp['label'] = label_dict[label]
        # Append the DataFrame to the list of DataFrames
        dfs.append(df_temp)
    else:
        print(f"Label for file {filename} not found in label_dict.")

# Concatenate all DataFrames in the list into one DataFrame
if dfs:
    df_valid = pd.concat(dfs, ignore_index=True)
else:
    df_valid = pd.DataFrame()

In [ ]:
df_valid.head()

,id,question,answer,label
0,synpic54733,what imaging modality was used to take this im...,ct with iv contrast,0
1,synpic25647,what kind of scan is this?,xr - plain film,0
2,synpic35681,was the mri taken with contrast?,no,0
3,synpic39641,"is this a t1 weighted, t2 weighted, or flair i...",t2,0
4,synpic35693,is this a noncontrast mri?,yes,0


In [ ]:
df_train = df_train[(df_train['label'] == 1)]
df_train.head()

,id,question,answer,label
6392,synpic41148,which plane is this image taken?,axial,1
6393,synpic43984,which plane is this image taken?,axial,1
6394,synpic38930,what plane is this ultrasound in?,longitudinal,1
6395,synpic52143,which plane is the image shown in?,coronal,1
6396,synpic20934,in what plane is this x-ray?,lateral,1


In [ ]:
df_valid = df_valid[(df_valid['label'] == 1)]
df_valid.head()

,id,question,answer,label
500,synpic54733,in what plane is this image oriented?,axial,1
501,synpic25647,what plane is this x-ray in?,ap,1
502,synpic35681,what plane is demonstrated?,coronal,1
503,synpic39641,in what plane was this image taken?,axial,1
504,synpic35693,in what plane is this mri taken?,sagittal,1


In [ ]:
labelencoder_train = LabelEncoder()
df_train['plane_type'] = labelencoder_train.fit_transform(df_train['answer'])
df_train.head()

labelencoder_valid = LabelEncoder()
df_valid['plane_type'] = labelencoder_valid.fit_transform(df_valid['answer'])
df_valid.head()

<ipython-input-10-7ba1f1edf945>:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_valid['plane_type'] = labelencoder_valid.fit_transform(df_valid['answer'])


,id,question,answer,label,plane_type
500,synpic54733,in what plane is this image oriented?,axial,1,2
501,synpic25647,what plane is this x-ray in?,ap,1,1
502,synpic35681,what plane is demonstrated?,coronal,1,3
503,synpic39641,in what plane was this image taken?,axial,1,2
504,synpic35693,in what plane is this mri taken?,sagittal,1,13


In [ ]:
integer_mapping = {labelencoder_valid.classes_[i]: i for i in range(len(labelencoder_valid.classes_))}
integer_mapping

{'3d reconstruction': 0,
 'ap': 1,
 'axial': 2,
 'coronal': 3,
 'frontal': 4,
 'lateral': 5,
 'longitudinal': 6,
 'mammo - cc': 7,
 'mammo - mag cc': 8,
 'mammo - mlo': 9,
 'mammo - xcc': 10,
 'oblique': 11,
 'pa': 12,
 'sagittal': 13,
 'transverse': 14}

In [ ]:
class ImageDataSet(Dataset):
    def __init__(self, image_names,plane_types, root_dir, transform=None):
        self.image_names = image_names
        self.plane_types = plane_types
        self.root_dir = root_dir
        self.transform = transform

    def __len__(self):
        return len(self.image_names)

    def __getitem__(self, idx):
        if torch.is_tensor(idx):
            idx = idx.tolist()
        img_name = '{}.jpg'.format(self.image_names[idx])
        fullname = join(self.root_dir, img_name)
        image = Image.open(fullname)
        labels = self.plane_types[idx]
        if self.transform:
            image = self.transform(image)
        return [image, labels]

In [ ]:
# Define transforms for image preprocessing
data_transforms = {
    'train': transforms.Compose([
        transforms.Resize(256),
        transforms.CenterCrop(224),
        transforms.ToTensor(),
        transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
    ]),
    'val': transforms.Compose([
        transforms.Resize(256),
        transforms.CenterCrop(224),
        transforms.ToTensor(),
        transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
    ]),
}

# Create datasets for training and validation
train_dataset = ImageDataSet(df_train['id'].values, df_train['plane_type'].values, TRAIN_FOLDER, transform=data_transforms['train'])
val_dataset = ImageDataSet(df_valid['id'].values, df_valid['plane_type'].values, VAL_FOLDER, transform=data_transforms['val'])

# Create data loaders
train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True)
val_loader = DataLoader(val_dataset, batch_size=32, shuffle=False)

In [ ]:
plane_trans = transforms.Compose([transforms.Resize(224),
                                  transforms.CenterCrop(224),
                               transforms.ToTensor(),
                               transforms.Normalize([0.485, 0.456, 0.406],
                             [0.229, 0.224, 0.225])])

# Aumentar batch?
train_ds = ImageDataSet(df_train['id'].values,df_train['plane_type'].values, TRAIN_FOLDER, transform=plane_trans)
train_dl = DataLoader(train_ds, batch_size=32, shuffle=True, num_workers=1)

valid_ds = ImageDataSet(df_valid['id'].values,df_valid['plane_type'].values, VAL_FOLDER, transform=plane_trans)
valid_dl = DataLoader(valid_ds, batch_size=32, shuffle=True, num_workers=1)

examples = next(iter(train_dl))

for label, img  in enumerate(examples):
   plt.imshow(img.permute(1,2,0))
   plt.show()
   print(f"Label: {label}")

In [ ]:
train_on_gpu = cuda.is_available()

In [ ]:
def get_pretrained_model(model_name):

    if  model_name == 'resnet50':
        model = models.resnet50(pretrained=True)

        for param in model.parameters():
            param.requires_grad = False

        n_inputs = model.fc.in_features
        model.fc = torch.nn.Sequential(
            torch.nn.Linear(n_inputs, 256), torch.nn.ReLU(), torch.nn.Dropout(0.2),
            torch.nn.Linear(256, 15), torch.nn.LogSoftmax(dim=1))

    # Move to gpu and parallelize
    if train_on_gpu:
        model = model.to('cuda')

    return model

In [ ]:
model = get_pretrained_model("resnet50")

/usr/local/lib/python3.10/dist-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=ResNet50_Weights.IMAGENET1K_V1`. You can also use `weights=ResNet50_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)
Downloading: "https://download.pytorch.org/models/resnet50-0676ba61.pth" to /root/.cache/torch/hub/checkpoints/resnet50-0676ba61.pth
100%|██████████| 97.8M/97.8M [00:01<00:00, 88.5MB/s]


In [ ]:
criterion = torch.nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)

In [ ]:
def train(model,
          criterion,
          optimizer,
          train_loader,
          valid_loader,
          nn.Module,
          max_epochs_stop=3,
          n_epochs=20,
          print_every=2):
    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
    print(f"Training on {device}")

    model.to(device)
    criterion.to(device)

    # Early stopping intialization
    epochs_no_improve = 0
    valid_loss_min = np.Inf

    valid_max_acc = 0
    history = []

    # Number of epochs already trained (if using loaded in model weights)
    try:
        print(f'Model has been trained for: {model.epochs} epochs.\n')
    except:
        model.epochs = 0
        print(f'Starting Training from Scratch.\n')

    overall_start = timer()

    # Main loop
    for epoch in range(n_epochs):

        # keep track of training and validation loss each epoch
        train_loss = 0.0
        valid_loss = 0.0

        train_acc = 0
        valid_acc = 0

        # Set to training
        model.train()
        start = timer()

        # Training loop
        for ii, (data, target) in enumerate(train_loader):
            print("model loop", ii)
            # Tensors to gpu
            if train_on_gpu:
                data, target = data.cuda(), target.cuda()

            # Clear gradients
            optimizer.zero_grad()
            # Predicted outputs are log probabilities
            output = model(data)

            # Loss and backpropagation of gradients
            loss = criterion(output, target)
            loss.backward()

            # Update the parameters
            optimizer.step()

            # Track train loss by multiplying average loss by number of examples in batch
            train_loss += loss.item() * data.size(0)

            # Calculate accuracy by finding max log probability
            _, pred = torch.max(output, dim=1)
            correct_tensor = pred.eq(target.data.view_as(pred))
            # Need to convert correct tensor from int to float to average
            accuracy = torch.mean(correct_tensor.type(torch.FloatTensor))
            # Multiply average accuracy times the number of examples in batch
            train_acc += accuracy.item() * data.size(0)

            # Track training progress
            print(
                f'Epoch: {epoch}\t{100 * (ii + 1) / len(train_loader):.2f}% complete. {timer() - start:.2f} seconds elapsed in epoch.',
                end='\r')

        # After training loops ends, start validation
        else:
            model.epochs += 1

            # Don't need to keep track of gradients
            with torch.no_grad():
                # Set to evaluation mode
                model.eval()

                # Validation loop
                for data, target in valid_loader:
                    # Tensors to gpu
                    if train_on_gpu:
                        data, target = data.cuda(), target.cuda()

                    # Forward pass
                    output = model(data)

                    # Validation loss
                    loss = criterion(output, target)
                    # Multiply average loss times the number of examples in batch
                    valid_loss += loss.item() * data.size(0)

                    # Calculate validation accuracy
                    _, pred = torch.max(output, dim=1)
                    correct_tensor = pred.eq(target.data.view_as(pred))
                    accuracy = torch.mean(
                        correct_tensor.type(torch.FloatTensor))
                    # Multiply average accuracy times the number of examples
                    valid_acc += accuracy.item() * data.size(0)

                # Calculate average losses
                train_loss = train_loss / len(train_loader.dataset)
                valid_loss = valid_loss / len(valid_loader.dataset)

                # Calculate average accuracy
                train_acc = train_acc / len(train_loader.dataset)
                valid_acc = valid_acc / len(valid_loader.dataset)

                history.append([train_loss, valid_loss, train_acc, valid_acc])

                # Print training and validation results
                if (epoch + 1) % print_every == 0:
                    print(
                        f'\nEpoch: {epoch} \tTraining Loss: {train_loss:.4f} \tValidation Loss: {valid_loss:.4f}'
                    )
                    print(
                        f'\t\tTraining Accuracy: {100 * train_acc:.2f}%\t Validation Accuracy: {100 * valid_acc:.2f}%'
                    )

                # Save the model if validation loss decreases
                if valid_loss < valid_loss_min:
                    # Save model
                    torch.save(model.state_dict(), save_file_name)
                    # Track improvement
                    epochs_no_improve = 0
                    valid_loss_min = valid_loss
                    valid_best_acc = valid_acc
                    best_epoch = epoch

                # Otherwise increment count of epochs with no improvement
                else:
                    epochs_no_improve += 1
                    # Trigger early stopping
                    if epochs_no_improve >= max_epochs_stop:
                        print(
                            f'\nEarly Stopping! Total epochs: {epoch}. Best epoch: {best_epoch} with loss: {valid_loss_min:.2f} and acc: {100 * valid_acc:.2f}%'
                        )
                        total_time = timer() - overall_start
                        print(
                            f'{total_time:.2f} total seconds elapsed. {total_time / (epoch+1):.2f} seconds per epoch.'
                        )

                        # Load the best state dict
                        model.load_state_dict(torch.load(save_file_name))
                        # Attach the optimizer
                        model.optimizer = optimizer

                        # Format history
                        history = pd.DataFrame(
                            history,
                            columns=[
                                'train_loss', 'valid_loss', 'train_acc',
                                'valid_acc'
                            ])
                        return model, history

    # Attach the optimizer
    model.optimizer = optimizer
    # Record overall time and print out stats
    total_time = timer() - overall_start
    print(
        f'\nBest epoch: {best_epoch} with loss: {valid_loss_min:.2f} and acc: {100 * valid_acc:.2f}%'
    )
    print(
        f'{total_time:.2f} total seconds elapsed. {total_time / (epoch):.2f} seconds per epoch.'
    )
    # Format history
    history = pd.DataFrame(
        history,
        columns=['train_loss', 'valid_loss', 'train_acc', 'valid_acc'])
    return model, history

In [ ]:
trained_model, history = train(model, criterion, optimizer, train_dl, valid_dl, 'best_model.pt', max_epochs_stop=5, n_epochs=30, print_every=2)

Training on cuda
Starting Training from Scratch.

model loop 0
model loop 1
model loop 2
model loop 3
model loop 4
model loop 5
model loop 6
model loop 7
model loop 8
model loop 9
model loop 10
model loop 11
model loop 12
model loop 13
model loop 14
model loop 15
model loop 16
model loop 17
model loop 18
model loop 19
model loop 20
model loop 21
model loop 22
model loop 23
model loop 24
model loop 25
model loop 26
model loop 27
model loop 28
model loop 29
model loop 30
model loop 31
model loop 32
model loop 33
model loop 34
model loop 35
model loop 36
model loop 37
model loop 38
model loop 39
model loop 40
model loop 41
model loop 42
model loop 43
model loop 44
model loop 45
model loop 46
model loop 47
model loop 48
model loop 49
model loop 50
model loop 51
model loop 52
model loop 53
model loop 54
model loop 55
model loop 56
model loop 57
model loop 58
model loop 59
model loop 60
model loop 61
model loop 62
model loop 63
model loop 64
model loop 65
model loop 66
model loop 67
model lo

In [ ]:
torch.save(trained_model.state_dict(), 'plane-model.pt')